# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Breast Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Lung_Adenocarcinoma_(LUAD)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Pancreatic_Cancer_(PAAD)']

If no match is found, jump directly to GEO in Part 2.2

In [4]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

cohort_dir = os.path.join(DATA_ROOT, dataset, trait_subdir)
clinical_data_file, genetic_data_file = xena_get_relevant_filepaths(cohort_dir)

In [5]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [6]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [7]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', '

In [8]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']


Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [9]:
preview_df(clinical_data[candidate_age_cols])

{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan],
 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0],
 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0],
 'year_of_initial_pathologic_diagnosis': [2004.0,
  2003.0,
  2011.0,
  2011.0,
  2013.0]}

In [10]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [11]:
preview_df(clinical_data[candidate_gender_cols])

{'Gender_nature2012': [nan, nan, nan, nan, nan],
 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [12]:
gender_col = 'gender'

In [13]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [14]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [15]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Breast Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-1.331386,-0.086682,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,0.360373,1.067905,0.076267
TCGA-3C-AALI-01,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,1.426614,-0.086682,0.312922,0.079683,0.475267,1.99661,0.161423,0.644673,-0.428695,0.068667
TCGA-3C-AALJ-01,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,1.694014,-0.086682,-0.748878,-0.656117,-0.216733,0.21081,0.101023,1.295073,0.915105,0.168567
TCGA-3C-AALK-01,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,-1.211186,-0.086682,0.415422,-0.178417,-0.211233,-1.28139,0.108023,0.496773,0.240105,3.099767
TCGA-4H-AAAK-01,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,-0.901186,0.424818,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,0.225573,-0.301995,0.200067


In [16]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

The merged dataset contains 1215 samples.
For the feature 'Breast Cancer', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Breast Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [17]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [18]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Haoyang/Breast-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [19]:
dataset = 'GEO'
trait_subdir = "Breast-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE243963', 'GSE243966', 'GSE248830', 'GSE249377']

Repeat the below steps for all the accession numbers

In [20]:
cohort = accession_num = "GSE248830"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/Breast-Cancer/GSE248830/GSE248830_family.soft.gz',
 '/media/techt/DATA/GEO/Breast-Cancer/GSE248830/GSE248830_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [21]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Unlocking Molecular mechanisms and identifying druggable targets in matched-paired brain metastasis of Breast and Lung cancers "
!Series_summary	"Introduction: The incidence of brain metastases in cancer patients is increasing, with lung and breast cancer being the most common sources. Despite advancements in targeted therapies, the prognosis remains poor, highlighting the importance to investigate the underlying mechanisms in brain metastases. The aim of this study was to investigate the differences in the molecular mechanisms involved in brain metastasis of breast and lung cancers. In addition, we aimed to identify cancer lineage-specific druggable targets in the brain metastasis. Methods: To that aim, a cohort of 44 FFPE tissue samples, including 22 breast cancer and 22 lung adenocarcinoma (LUAD) and their matched-paired brain metastases were collected. Targeted gene expression profiles of primary tumors were compared to their matched-paired brain metastases samples u

In [22]:
clinical_data.head()

,!Sample_geo_accession,GSM7920782,GSM7920783,GSM7920784,GSM7920785,GSM7920786,GSM7920787,GSM7920788,GSM7920789,GSM7920790,...,GSM7920816,GSM7920817,GSM7920818,GSM7920819,GSM7920820,GSM7920821,GSM7920822,GSM7920823,GSM7920824,GSM7920825
0,!Sample_characteristics_ch1,age at diagnosis: 49,age at diagnosis: 44,age at diagnosis: 41,age at diagnosis: 40,age at diagnosis: 48,age at diagnosis: 42,age at diagnosis: 47,age at diagnosis: 53,age at diagnosis: 41,...,age at diagnosis: 63,age at diagnosis: 60,age at diagnosis: 58,age at diagnosis: 70,age at diagnosis: 66,age at diagnosis: 64,age at diagnosis: 60,age at diagnosis: 50,age at diagnosis: 66,age at diagnosis: 74
1,!Sample_characteristics_ch1,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,...,Sex: male,Sex: male,Sex: female,Sex: female,Sex: female,Sex: male,Sex: male,Sex: female,Sex: male,Sex: male
2,!Sample_characteristics_ch1,histology: TNBC,histology: ER+ PR+ HER2-,histology: Unknown,histology: ER+ PR+ HER2-,histology: TNBC,histology: Unknown,histology: TNBC,histology: ER- PR- HER2+,histology: TNBC,...,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma,histology: adenocaricnoma
3,!Sample_characteristics_ch1,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,smoking status: n.a,...,smoking status: former-smoker,smoking status: former-smoker,smoking status: smoker,smoking status: Never smoking,smoking status: smoker,smoking status: smoker,smoking status: former-smoker,smoking status: unknown,smoking status: smoker,smoking status: former-roker
4,!Sample_characteristics_ch1,treatment after surgery of bm: surgery + chemo...,treatment after surgery of bm: surgery + chem...,treatment after surgery of bm: surgery + chemo...,treatment after surgery of bm: surgery,treatment after surgery of bm: surgery,treatment after surgery of bm: surgery + chem...,treatment after surgery of bm: surgery + chemo...,treatment after surgery of bm: surgery + chemo...,treatment after surgery of bm: surgery + chemo...,...,treatment after surgery of bm: Radiotherapy,treatment after surgery of bm: Radiotherapy,treatment after surgery of bm: Radiotherapy,treatment after surgery of bm: No treatment,treatment after surgery of bm: Radiotherapy,treatment after surgery of bm: WBRT,treatment after surgery of bm: SRT,treatment after surgery of bm: WBRT,treatment after surgery of bm: Radiotherapy,treatment after surgery of bm: Radiotherapy


In [23]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['age at diagnosis: 49',
  'age at diagnosis: 44',
  'age at diagnosis: 41',
  'age at diagnosis: 40',
  'age at diagnosis: 48',
  'age at diagnosis: 42',
  'age at diagnosis: 47',
  'age at diagnosis: 53',
  'age at diagnosis: 74',
  'age at diagnosis: 58',
  'age at diagnosis: 51',
  'age at diagnosis: 55',
  'age at diagnosis: 46',
  'age at diagnosis: 59',
  'age at diagnosis: 50',
  'age at diagnosis: 57',
  'age at diagnosis: 60',
  'age at diagnosis: 69',
  'age at diagnosis: n.a.',
  'age at diagnosis: 65',
  'age at diagnosis: 37',
  'age at diagnosis: 63',
  'age at diagnosis: 70',
  'age at diagnosis: 66',
  'age at diagnosis: 64'],
 1: ['Sex: female', 'Sex: male'],
 2: ['histology: TNBC',
  'histology: ER+ PR+ HER2-',
  'histology: Unknown',
  'histology: ER- PR- HER2+',
  'histology: ER+ PR-HER2+',
  'histology: ER+ PR- HER2-',
  'histology: ER- PR+ HER2-',
  'histology: adenocaricnoma'],
 3: ['smoking status: n.a',
  'smoking status: former-smoker',
  'smoking status:

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [24]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Breast Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Breast Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is a

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [25]:
age_row = gender_row = None
convert_age = convert_gender = None

In [26]:
is_gene_availabe = True
trait_row = 2
age_row = 0
gender_row = 1

trait_type = 'binary'

In [27]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [28]:
# Verify and use the functions generated by GPT

def convert_trait(histology_string):
    """
    Convert histology string to a binary value representing breast cancer.
    Presence of breast cancer is represented as 1, absence as 0.
    Unknown or non-breast cancer histologies are converted to None.
    """
    breast_cancer_histologies = ['tnbc', 'er+ pr+ her2-', 'er- pr- her2+', 'er+ pr-her2+', 'er+ pr- her2-',
                                 'er- pr+ her2-']
    histology = histology_string.split(': ')[1].lower()
    if histology in breast_cancer_histologies:
        return 1
    elif histology == 'unknown':
        return None
    else:
        return 0


def convert_age(age_string):
    """
    Convert age string to a continuous numerical value.
    Unknown values are converted to None.
    """
    if age_string.lower() == 'n.a.':
        return None
    try:
        # Extract age as an integer from the string
        age = int(age_string.split(': ')[1])
        return age
    except (ValueError, IndexError):
        # In case of any format error or unexpected string structure
        return None


# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(gender_string):
    """
    Convert gender string to a binary value.
    'female' is represented as 1, 'male' as 0.
    Unknown values are converted to None.
    """
    if gender_string.lower() == 'sex: female':
        return 1
    elif gender_string.lower() == 'sex: male':
        return 0
    else:
        return None

In [29]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM7920782,GSM7920783,GSM7920784,GSM7920785,GSM7920786,GSM7920787,GSM7920788,GSM7920789,GSM7920790,GSM7920791,...,GSM7920816,GSM7920817,GSM7920818,GSM7920819,GSM7920820,GSM7920821,GSM7920822,GSM7920823,GSM7920824,GSM7920825
Breast Cancer,1,1,None,1,1,None,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Age,49,44,41,40,48,42,47,53,41,74,...,63,60,58,70,66,64,60,50,66,74
Gender,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,0,0,1,0,0


### Genetic data preprocessing and final filtering

In [30]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM7920782,GSM7920783,GSM7920784,GSM7920785,GSM7920786,GSM7920787,GSM7920788,GSM7920789,GSM7920790,GSM7920791,...,GSM7920816,GSM7920817,GSM7920818,GSM7920819,GSM7920820,GSM7920821,GSM7920822,GSM7920823,GSM7920824,GSM7920825
ID,,,,,,,,,,,,,,,,,,,,,
A2M,13.210102,13.237969,14.728878,15.399663,12.970464,13.014807,12.521967,13.055481,13.219538,14.543370,...,12.635802,12.487090,11.743261,13.219376,12.499513,11.719452,12.509194,11.682523,10.885632,11.277235
ACVR1C,5.128485,5.336850,5.611523,5.041711,6.342262,5.005301,4.148600,5.665523,5.548821,5.081208,...,5.756039,6.795792,6.873777,6.848304,5.106051,6.790117,3.936121,2.998025,6.339292,4.463217
ADAM12,9.806557,12.374397,9.953351,8.507092,11.402156,10.739489,9.114680,11.095729,8.762302,9.600964,...,8.280338,6.973884,7.113936,8.036966,8.398832,5.346283,9.429915,7.863096,10.774084,4.708329
ADGRE1,4.128485,3.751888,4.611523,4.582279,3.771514,4.420338,2.863198,4.800452,3.860765,3.844169,...,2.806079,5.926579,2.066422,6.447342,4.286623,3.665105,3.415288,1.190670,2.251830,-0.291671
ADM,10.953762,10.544678,8.155844,9.333824,8.438938,9.446117,8.285262,8.551518,9.567357,8.437694,...,11.577869,11.051458,7.742883,9.596892,6.993576,8.531638,8.579977,8.150672,9.623969,7.790478


In [31]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['A2M',
 'ACVR1C',
 'ADAM12',
 'ADGRE1',
 'ADM',
 'ADORA2A',
 'AKT1',
 'ALDOA',
 'ALDOC',
 'ANGPT1',
 'ANGPT2',
 'ANGPTL4',
 'ANLN',
 'APC',
 'APH1B',
 'API5',
 'APLNR',
 'APOE',
 'APOL6',
 'AQP9']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [32]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A2M', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B', 'API5', 'APLNR', 'APOE', 'APOL6', 'AQP9']\n"


If not required, jump directly to the gene normalization step

In [33]:
requires_gene_mapping = False

In [34]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [35]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [36]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [37]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [38]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [39]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 38 samples.


In [40]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Breast Cancer', the least common label is '1.0' with 19 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Breast Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 47.25
  50% (Median): 55.0
  75%: 62.25
Min: 37.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 11 occurrences. This represents 28.95% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [41]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [42]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [43]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1215,
1,GSE248830,True,True,False,True,True,38,


In [44]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'Xena'

In [45]:

condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1215,
1,GSE248830,True,True,False,True,True,38,


In [46]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Breast Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-1.331386,-0.086682,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,0.360373,1.067905,0.076267
1,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,1.426614,-0.086682,0.312922,0.079683,0.475267,1.99661,0.161423,0.644673,-0.428695,0.068667
2,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,1.694014,-0.086682,-0.748878,-0.656117,-0.216733,0.21081,0.101023,1.295073,0.915105,0.168567
3,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,-1.211186,-0.086682,0.415422,-0.178417,-0.211233,-1.28139,0.108023,0.496773,0.240105,3.099767
4,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,-0.901186,0.424818,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,0.225573,-0.301995,0.200067


In [47]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values




Select the appropriate regression model depending on whether the dataset shows batch effect.

In [48]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [49]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [50]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 67.82% ± 6.72%


In [51]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 90.86% ± 0.84%


In [52]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 4. Discussion and report

In [53]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0065
Found 4 genes with non-zero coefficients associated with the trait 'Breast Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
    Variable  Coefficient  Absolute Coefficient
     C5orf52    -0.503090              0.503090
LOC100129935    -0.091620              0.091620
        HAO2     0.043399              0.043399
      TTTY21    -0.008845              0.008845


/home/techt/AI4Science_gold_standard/utils.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [54]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0065
Found 56 significant genes associated with the trait 'Breast Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
 Variable  Coefficient      p_value  corrected_p_value
  COL10A1     0.139801 2.272959e-32       3.941083e-28
      BGN     0.096090 1.371919e-18       1.189385e-14
    MMP11     0.082105 1.200864e-14       6.940595e-11
  HSD17B6     0.051075 5.396057e-11       2.099223e-07
   CTHRC1     0.058965 6.696716e-11       2.099223e-07
    MFAP5    -0.055412 7.264165e-11       2.099223e-07
   OR51D1     0.023976 1.231414e-09       3.050213e-06
   CLEC5A     0.046623 1.873539e-09       4.060661e-06
       GC     0.027701 5.093728e-09       9.813350e-06
      FN1     0.064388 3.437602e-08       5.960458e-05
  C1QTNF6     0.050616 5.341210e-08       7.717604e-05
     COMP     0.040724 5.254171e-08       7.717604e-05
   LRRC15     0.051955 6.597041e-08       8.798931e-05
    INHBA     0

/home/techt/AI4Science_gold_standard/utils.py:495: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
